# Basic functioning

In [1]:
import urllib3
import pickle

In [2]:
def functioncall(fn_input):
    request = functioncall.PoolManager.request('POST', 'http://127.0.0.1:1234', body = pickle.dumps(fn_input))
    return pickle.loads(request.data)
setattr(functioncall, 'PoolManager', urllib3.PoolManager())

In [3]:
functioncall("a test string")

'gnirts tset aA GLOBAL DEPENDENCY'

# TESTING

In [4]:
%%timeit
_ = functioncall("a test string")

303 ms ± 701 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Check with large inputs

In [5]:
import time
import sys

In [6]:
large_input = "very very very large text" * 20000
first = time.perf_counter()
fn_output = functioncall(large_input)
second = time.perf_counter()
print("Verify output", fn_output == large_input[::-1] + "A GLOBAL DEPENDENCY")
print("time taken", round((second - first),5))
print("0.5 MB input and output", sys.getsizeof(large_input),sys.getsizeof(fn_output))

Verify output True
time taken 0.31254
0.5 MB input and output 500049 500068


## Check with many concurrent inputs

In [7]:
from multiprocessing import Pool
from adv_functioncall import *

textlist = ["FIRST TEST", "second test" , "long test"*1000] * 8

#first = time.perf_counter() Inaccurate timings

nprocesses = 12 #can go up to socketserver.TCPServer.request_queue_size + 1 without error
with Pool(nprocesses) as p:
    result = p.map(len, textlist) #to start the pool
    first = time.perf_counter() #Better timings
    result = p.map(adv_functioncall, textlist)
    
second = time.perf_counter()

time_taken = round((second - first),5)
print(time_taken)
print(sorted([round(item[1],1) for item in result])) #To show the effects of single threaded server

7.52865
[0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3.0, 3.3, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6]
